In [ ]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, Border, Side, Alignment, PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows
import threading
import os

class ColumnConfigWindow(tk.Toplevel):
    def __init__(self, parent, app, columns):
        super().__init__(parent)
        self.app = app
        self.columns = columns
        self.selected_columns = []
        
        self.create_widgets()
        self.load_columns()
        
    def create_widgets(self):
        self.title("Выбор столбцов для отчета")
        self.geometry("600x400")
        
        main_frame = ttk.Frame(self)
        main_frame.pack(padx=10, pady=10, fill='both', expand=True)
        
        # Список доступных столбцов с прокруткой
        columns_frame = ttk.Frame(main_frame)
        columns_frame.pack(fill='both', expand=True)
        
        ttk.Label(columns_frame, text="Доступные столбцы:").pack(anchor='w')
        
        self.columns_list = ttk.Treeview(columns_frame, show='tree', selectmode='extended')
        scroll_y = ttk.Scrollbar(columns_frame, orient='vertical', command=self.columns_list.yview)
        self.columns_list.configure(yscrollcommand=scroll_y.set)
        
        self.columns_list.pack(side='left', fill='both', expand=True)
        scroll_y.pack(side='right', fill='y')
        
        # Кнопки управления
        btn_frame = ttk.Frame(main_frame)
        btn_frame.pack(pady=10)
        
        ttk.Button(btn_frame, text="Сохранить", command=self.save_config).pack(side='right', padx=5)
        ttk.Button(btn_frame, text="Отмена", command=self.destroy).pack(side='right')

    def load_columns(self):
        """Загрузка столбцов в список"""
        if not self.columns:
            messagebox.showwarning("Предупреждение", "В файле нет доступных столбцов")
            return
            
        for col in self.columns:
            self.columns_list.insert('', 'end', text=col, values=(col,))

    def save_config(self):
        """Сохранение конфигурации"""
        self.selected_columns = [
            self.columns_list.item(item)['values'][0] 
            for item in self.columns_list.selection()
        ]
        
        if not self.selected_columns:
            messagebox.showwarning("Предупреждение", "Выберите хотя бы один столбец")
            return
            
        self.app.column_config = self.selected_columns
        self.destroy()

class ReportApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Генератор отчетов")
        self.root.geometry("800x600")
        
        self.main_file = tk.StringVar()
        self.sku_file = tk.StringVar()
        self.tt_file = tk.StringVar()
        self.column_config = None
        
        self.create_widgets()
        self.setup_styles()
        
    def setup_styles(self):
        self.style = ttk.Style()
        self.style.configure('TButton', padding=6)
        self.style.configure('TFrame', padding=10)
        self.style.configure('Header.TLabel', font=('Helvetica', 10, 'bold'))
        
    def create_widgets(self):
        main_frame = ttk.Frame(self.root)
        main_frame.pack(padx=10, pady=10, fill='both', expand=True)
        
        # Блок выбора файлов
        file_frame = ttk.LabelFrame(main_frame, text="1. Выбор файлов")
        file_frame.pack(fill='x', pady=5)
        
        self.create_file_row(file_frame, "Исходная таблица:", self.main_file, 0)
        self.create_file_row(file_frame, "Справочник SKU:", self.sku_file, 1)
        self.create_file_row(file_frame, "Справочник ТТ:", self.tt_file, 2)
        
        # Блок настройки
        config_frame = ttk.LabelFrame(main_frame, text="2. Настройка отчета")
        config_frame.pack(fill='x', pady=5)
        
        ttk.Button(config_frame, 
                 text="Выбрать столбцы", 
                 command=self.show_column_config).pack(pady=5)
        
        # Прогресс бар
        self.progress = ttk.Progressbar(main_frame, orient='horizontal', length=300, mode='determinate')
        self.progress.pack(pady=10)
        
        # Лог выполнения
        self.log_text = tk.Text(main_frame, height=15, state="disabled")
        self.log_text.pack(fill='both', expand=True)
        
        # Кнопка запуска
        self.run_btn = ttk.Button(main_frame, 
                                text="3. Создать отчет", 
                                command=self.start_processing)
        self.run_btn.pack(pady=10)
        
    def create_file_row(self, parent, label_text, target_var, row):
        frame = ttk.Frame(parent)
        frame.grid(row=row, column=0, sticky='ew', pady=2)
        
        label = ttk.Label(frame, text=label_text, width=20)
        label.pack(side='left', padx=5)
        
        entry = ttk.Entry(frame, textvariable=target_var, width=60)
        entry.pack(side='left', fill='x', expand=True)
        
        btn = ttk.Button(frame, text="Обзор", command=lambda: self.select_file(target_var))
        btn.pack(side='left', padx=5)

    def select_file(self, target_var):
        file_path = filedialog.askopenfilename(
            filetypes=[
                ("Excel files", "*.xlsx *.xlsb"), 
                ("All files", "*.*")
            ]
        )
        if file_path:
            target_var.set(file_path)
            
    def log_message(self, message):
        self.log_text.config(state="normal")
        self.log_text.insert("end", message + "\n")
        self.log_text.see("end")
        self.log_text.config(state="disabled")
    
    def update_progress(self, value):
        self.progress["value"] = value
        self.root.update_idletasks()
        
    def start_processing(self):
        if not all([self.main_file.get(), self.sku_file.get(), self.tt_file.get()]):
            messagebox.showerror("Ошибка", "Необходимо выбрать все файлы!")
            return
            
        thread = threading.Thread(target=self.create_report)
        thread.start()
        
    def read_excel_file(self, file_path):
        try:
            _, ext = os.path.splitext(file_path)
            engine = 'pyxlsb' if ext.lower() == '.xlsb' else 'openpyxl'
            return pd.read_excel(file_path, engine=engine)
        except Exception as e:
            self.log_message(f"Ошибка чтения файла {file_path}: {str(e)}")
            raise
        
    def show_column_config(self):
        if not self.main_file.get():
            messagebox.showwarning("Предупреждение", "Сначала загрузите основной файл")
            return
            
        try:
            df = self.read_excel_file(self.main_file.get())
            if df.empty:
                messagebox.showerror("Ошибка", "Файл не содержит данных")
                return
                
            config_window = ColumnConfigWindow(self.root, self, df.columns.tolist())
            self.root.wait_window(config_window)
        except Exception as e:
            messagebox.showerror("Ошибка", str(e))

    def create_report(self):
        try:
            self.run_btn.config(state="disabled")
            self.log_message("Начало обработки данных...")
            
            # Загрузка данных
            self.log_message("Загрузка основной таблицы...")
            df_main = self.read_excel_file(self.main_file.get())
            self.update_progress(20)
            
            self.log_message("Загрузка справочника SKU...")
            df_sku = self.read_excel_file(self.sku_file.get())
            self.update_progress(40)
            
            self.log_message("Загрузка справочника ТТ...")
            df_tt = self.read_excel_file(self.tt_file.get())
            self.update_progress(60)
            
            # Фильтрация выбранных столбцов
            if self.column_config:
                self.log_message("Фильтрация выбранных столбцов...")
                available_columns = [col for col in self.column_config if col in df_main.columns]
                df_main = df_main[available_columns + ['Реализация шт.(кг)', 'Реализация Сумма', 'Себестоимость(руб.)']]

            # Обработка данных
            self.log_message("Агрегация данных...")
            group_cols = ['Месяц', 'Год', '№ магазина', 'Код товара', 'Наименование товара']
            numeric_cols = ['Реализация шт.(кг)', 'Реализация Сумма', 'Себестоимость(руб.)']
            
            df_month = df_main.groupby(group_cols)[numeric_cols].sum().reset_index()
            
            # Добавление справочной информации
            self.log_message("Обогащение данными из справочников...")
            df_month['Сеть'] = 'VERNYY'
            df_month['Глобальный код ТТ'] = 'VERNYY' + df_month['№ магазина'].astype(str)
            df_month['Глобальный Код SKU'] = 'VERNYY' + df_month['Код товара'].astype(str)
            
            sku_mapping = df_sku.set_index('Глобальный Код SKU')['Наименование SKU'].to_dict()
            df_month['Наименование SKU'] = df_month['Глобальный Код SKU'].map(sku_mapping)
            
            # Формирование финальной таблицы
            result_cols = [
                'Месяц', 'Год', 'Сеть', 'Глобальный код ТТ', 
                '№ магазина', 'Глобальный Код SKU', 'Наименование SKU',
                'Реализация шт.(кг)', 'Реализация Сумма', 'Себестоимость(руб.)'
            ]

            final_df = df_month[result_cols].rename(columns={
                '№ магазина': 'Код ТТ',
                'Реализация шт.(кг)': 'Продажа(шт)',
                'Реализация Сумма': 'Оборот(руб.)'
            })
            
            # Сохранение с форматированием
            self.log_message("Создание Excel с форматированием...")
            save_path = filedialog.asksaveasfilename(
                defaultextension=".xlsx",
                filetypes=[("Excel files", "*.xlsx"), ("All files", "*.*")]
            )
            
            if save_path:
                wb = Workbook()
                ws = wb.active
                ws.title = "Отчет VERNYY"
                
                for r in dataframe_to_rows(final_df, index=False, header=True):
                    ws.append(r)
                
                # Форматирование
                header_font = Font(bold=True, color="FFFFFF")
                header_fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")
                thin_border = Border(
                    left=Side(style='thin'), 
                    right=Side(style='thin'), 
                    top=Side(style='thin'), 
                    bottom=Side(style='thin')
                )
                
                for cell in ws[1]:
                    cell.font = header_font
                    cell.fill = header_fill
                    cell.border = thin_border
                    cell.alignment = Alignment(horizontal="center", vertical="center")
                
                for row in ws.iter_rows(min_row=2):
                    for cell in row:
                        cell.border = thin_border
                        cell.alignment = Alignment(horizontal="center", vertical="center")
                        
                        if cell.column_letter in ['H', 'I', 'J']:
                            cell.number_format = '#,##0.00'
                        elif cell.column_letter in ['A', 'B']:
                            cell.number_format = '0'
                
                # Автоподбор ширины
                for col in ws.columns:
                    max_length = max(len(str(cell.value)) for cell in col)
                    adjusted_width = (max_length + 2) * 1.2
                    ws.column_dimensions[col[0].column_letter].width = adjusted_width
                
                ws.freeze_panes = 'A2'
                
                wb.save(save_path)
                self.log_message(f"Файл успешно сохранен: {save_path}")
                messagebox.showinfo("Готово", "Отчет сформирован!")
            
            self.update_progress(100)
            
        except Exception as e:
            self.log_message(f"Критическая ошибка: {str(e)}")
            messagebox.showerror("Ошибка", str(e))
        finally:
            self.run_btn.config(state="normal")
            self.update_progress(0)

if __name__ == "__main__":
    root = tk.Tk()
    app = ReportApp(root)
    root.mainloop()

2025-03-31 22:14:40.109 python[91333:7529028] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-31 22:14:40.109 python[91333:7529028] +[IMKInputSession subclass]: chose IMKInputSession_Modern
2025-03-31 22:14:46.982 python[91333:7529028] The class 'NSOpenPanel' overrides the method identifier.  This method is implemented by class 'NSWindow'
